In [ ]:
import time
import random
import threading
from collections import deque
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

class WearableDevice:
    """
    Simulates a wearable IoT device that collects sensor data with activity level.
    """
    def __init__(self, device_id, sampling_rate=1):
        """
        Initializes the wearable device.
        Args:
            device_id (str): Unique identifier for the device.
            sampling_rate (int): The rate at which data is collected (in Hz).
        """
        self.device_id = device_id
        self.sampling_rate = sampling_rate
        self.heart_rate = 70
        self.oxygen_level = 98
        self.steps = 0
        self.battery_level = 100
        self.running = True
        self.data_queue = deque(maxlen=100)
        self.activity_level = "sedentary"

    def get_sensor_data(self):
        """
        Simulates getting sensor data, now influenced by activity level.
        """
        if not self.running:
            return None

        if self.activity_level == "sedentary":
            self.heart_rate += random.randint(-5, 5)
            self.heart_rate = max(60, min(self.heart_rate, 80))
            self.oxygen_level += random.uniform(-0.1, 0.1)
            self.oxygen_level = max(95, min(self.oxygen_level, 100))
            self.steps += random.randint(1, 5)
        elif self.activity_level == "walking":
            self.heart_rate += random.randint(5, 15)
            self.heart_rate = max(80, min(self.heart_rate, 120))
            self.oxygen_level += random.uniform(-0.5, 0.2)
            self.oxygen_level = max(90, min(self.oxygen_level, 99))
            self.steps += random.randint(10, 20)
        elif self.activity_level == "running":
            self.heart_rate += random.randint(10, 20)
            self.heart_rate = max(120, min(self.heart_rate, 180))
            self.oxygen_level += random.uniform(-1.0, 0.5)
            self.oxygen_level = max(85, min(self.oxygen_level, 98))
            self.steps += random.randint(20, 40)

        self.battery_level -= 0.1 if self.heart_rate < 120 else 0.5
        self.battery_level = max(0, self.battery_level)

        data = {
            "heart_rate": self.heart_rate,
            "oxygen_level": self.oxygen_level,
            "steps": self.steps,
            "battery_level": self.battery_level,
            "timestamp": time.time(),
            "activity_level": self.activity_level
        }
        self.data_queue.append(data)
        return data

    def run(self):
        """Simulates the device running."""
        self.running = True

    def stop(self):
        """Simulates the device being stopped."""
        self.running = False

    def get_data_history(self):
        """Returns the data history."""
        return list(self.data_queue)

    def update_activity_level(self, activity):
        """Updates the activity level of the device."""
        if activity in ["sedentary", "walking", "running"]:
            self.activity_level = activity
        else:
            raise ValueError(f"Invalid activity level: {activity}")

    def __repr__(self):
        return (f"WearableDevice(id={self.device_id}, heart_rate={self.heart_rate}, "
                f"oxygen_level={self.oxygen_level:.1f}, steps={self.steps}, "
                f"battery_level={self.battery_level:.1f}, running={self.running}, "
                f"activity_level={self.activity_level})")


class DataProcessor:
    """
    Simulates processing data from a wearable device and making predictions.
    """
    def __init__(self, device_id):
        """
        Initializes the data processor.
        Args:
            device_id (str): The ID of the wearable device.
        """
        self.device_id = device_id
        self.processed_data = []
        self.running = True
        self.model = None
        self.training_data = []

    def process_data(self, data):
        """
        Processes the sensor data and makes predictions.
        Args:
            data (dict): A dictionary containing the sensor data.
        """
        if not self.running or data is None:
            return

        health_score = (data["oxygen_level"] + (190 - data["heart_rate"]) / 2) * (
                data["battery_level"] / 100
        )
        processed_data = {
            "device_id": self.device_id,
            "timestamp": data['timestamp'],
            "heart_rate": data["heart_rate"],
            "oxygen_level": data["oxygen_level"],
            "steps": data["steps"],
            "battery_level": data["battery_level"],
            "health_score": health_score,
            "activity_level": data["activity_level"],
        }
        self.processed_data.append(processed_data)
        self.training_data.append(data)
        print(f"Processed data from {self.device_id}: {processed_data}")

        if self.model:
            prediction = self.predict_health_status(data)
            print(f"Predicted health status for {self.device_id}: {prediction}")

    def run(self):
        """Sets the processor to running."""
        self.running = True

    def stop(self):
        """Sets the processor to stop."""
        self.running = False

    def get_processed_data(self):
        """Returns all processed data"""
        return self.processed_data

    def get_recent_processed_data(self, num_last_entries=10):
        """Returns the last n processed data entries"""
        return self.processed_data[-num_last_entries:]

    def train_model(self, data):
        """
        Trains a machine learning model to predict health status.  This is a
        simplified example using a RandomForestRegressor.  You would likely use a
        more sophisticated model and feature engineering in a real application.
        Args:
            data (list): A list of data dictionaries.
        """
        if not data:
            print("Not enough data to train the model.")
            return

        df = pd.DataFrame(data)

        df['hr_zone'] = df['heart_rate'].apply(
            lambda hr: 'low' if hr < 80 else ('medium' if hr < 120 else 'high')
        )

        df['health_score'] = (df["oxygen_level"] + (190 - df["heart_rate"]) / 2) * (
                df["battery_level"] / 100
        )

        df = pd.get_dummies(df, columns=['activity_level', 'hr_zone'])

        features = [
            "heart_rate",
            "oxygen_level",
            "steps",
            "battery_level",
            'activity_level_sedentary', 'activity_level_walking', 'activity_level_running',
            'hr_zone_low', 'hr_zone_medium', 'hr_zone_high'
        ]
        target = "health_score"

        missing_features = [f for f in features if f not in df.columns]
        if missing_features:
            print(f"Warning: Missing features for training: {missing_features}")
            return

        X = df[features]
        y = df[target]
        # Split data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )  # 80% training, 20% testing

        # Train a Random Forest Regressor
        self.model = RandomForestRegressor(n_estimators=100, random_state=42)
        self.model.fit(X_train, y_train)

        # Evaluate the model on the test set
        y_pred = self.model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        print(f"Model trained. Mean Squared Error on test set: {mse:.2f}")

    def predict_health_status(self, data):
        """
        Predicts the health status based on the given data.
        Args:
            data (dict): A dictionary containing sensor data.
        Returns:
            str: Predicted health status ("Good", "Moderate", "Bad"), or None if no model.
        """
        if not self.model:
            return "No model trained yet."

        df = pd.DataFrame([data])
        # Feature engineering
        df['hr_zone'] = df['heart_rate'].apply(
            lambda hr: 'low' if hr < 80 else ('medium' if hr < 120 else 'high')
        )
        df = pd.get_dummies(df, columns=['activity_level', 'hr_zone'])

        missing_features = [f for f in self.model.feature_names_in_ if f not in df.columns]
        if missing_features:
            print(f"Warning: Missing features for prediction: {missing_features}")
            return "Missing data for prediction."

        X = df[self.model.feature_names_in_]

        prediction = self.model.predict(X)[0]

        # Map the prediction to a health status category
        if prediction > 80:
            return "Good"
        elif prediction > 60:
            return "Moderate"
        else:
            return "Bad"

    def retrain_model(self):
        """Retrains the model with all available training data."""
        print("Retraining model...")
        self.train_model(self.training_data)
        print("Model retrained.")


def simulate_data_flow(device, processor):
    """
    Simulates data flow from a wearable device to a data processor and triggers
    model retraining.
    Args:
        device (WearableDevice): The wearable device.
        processor (DataProcessor): The data processor.
    """
    while True:
        sensor_data = device.get_sensor_data()
        processor.process_data(sensor_data)
        time.sleep(device.sampling_rate)

if __name__ == "__main__":
    # Create a wearable device and a data processor
    wearable_device = WearableDevice("Device001", sampling_rate=1)
    data_processor = DataProcessor("Device001")

    # Start the data flow simulation in a separate thread
    data_thread = threading.Thread(
        target=simulate_data_flow, args=(wearable_device, data_processor,)
    )
    data_thread.daemon = True
    data_thread.start()

    # Simulate the device running with changing activity levels
    print(f"Initial Device State: {wearable_device}")
    time.sleep(5)
    wearable_device.update_activity_level("walking")
    print("Activity level changed to walking.")
    time.sleep(5)
    wearable_device.update_activity_level("running")
    print("Activity level changed to running.")
    time.sleep(5)
    wearable_device.stop()
    data_processor.stop()
    time.sleep(2)

    print(f"Final Device State: {wearable_device}")
    print(f"All processed data: {data_processor.get_processed_data()}")

    # Train the model after collecting some data
    print("Training the model...")
    data_processor.train_model(data_processor.training_data)

    # Simulate more data and re-train:
    wearable_device.run()
    time.sleep(10)
    wearable_device.stop()
    data_processor.stop()
    time.sleep(2)

    data_processor.retrain_model() #retrain model.

    print("Exiting...")


Processed data from Device001: {'device_id': 'Device001', 'timestamp': 1746422727.8554027, 'heart_rate': 74, 'oxygen_level': 98.05961247739565, 'steps': 3, 'battery_level': 99.9, 'health_score': 155.90355286491828, 'activity_level': 'sedentary'}Initial Device State: WearableDevice(id=Device001, heart_rate=74, oxygen_level=98.1, steps=3, battery_level=99.9, running=True, activity_level=sedentary)

Processed data from Device001: {'device_id': 'Device001', 'timestamp': 1746422728.8618355, 'heart_rate': 79, 'oxygen_level': 98.13852551784258, 'steps': 7, 'battery_level': 99.80000000000001, 'health_score': 153.33124846680693, 'activity_level': 'sedentary'}
Processed data from Device001: {'device_id': 'Device001', 'timestamp': 1746422729.8621602, 'heart_rate': 78, 'oxygen_level': 98.09441947242601, 'steps': 9, 'battery_level': 99.70000000000002, 'health_score': 153.63213621400877, 'activity_level': 'sedentary'}
Processed data from Device001: {'device_id': 'Device001', 'timestamp': 1746422730.